In [181]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from re import findall as fa
import sqlite3
import pymorphy2

from nltk.tokenize import word_tokenize

In [440]:
text = """
Иногда я пишу о книгах, которые произвели на меня впечатление. Писать большие отзывы сейчас не хочется, поэтому в порядке перечисления.
"Атлант расправил плечи" - за последнее время понравилась больше всего наряду с Довлатовым (но насчет последнего сомнений и не было). Почему-то раньше я думал, что это что-то вроде "Финансиста" Драйзера. Так же, видимо, думают и люди, рисующие мемы "сын маминой подруги расправил плечи". А на самом деле книга об альтернативной вселенной, где в США наступил социализм. Очень рекомендую.
Что до "Финансиста" Драйзера, то т.д. он надолго отбил у меня желание читать этого автора. Не потому что мне не интересно читать про рынок - наоборот, про рынок интересно. Но всё остальное там скучно, особенно герои. Может быть, так и было задумано, но я это не люблю.
Дилогия об Остапе Бендере - начинаются обе книги весело, кончаются обе книги уныло. Не столько с точки зрения событий, сколько с точки зрения того, как трансформируется язык. Поэтому от них остается неприятное ощущение, хотя написаны они ярко, весело и интересно. Впрочем, не пойти на такую сделку вряд ли можно было в условиях, в которых работали авторы.
"Три мушкетера". Ну, не побоюсь этого я слова, такое. Занятно, но не более того - я сейчас даже с трудом вспомнил об этой книжке. Главный интерес книжка представляет с исторической точки зрения. В том числе и потому, что является убедительным доказательством, что во Франции в 17м веке был интернет и портативные телепорты - ну или по крайней мере бесстыдная сценарная магия.
Отто Кариус, "Тигры в грязи". необходимость обязана Язык Все всяк по видимому каждому каждый каждая этой можетбыть кажеться наверное наверно книги совершенно ужасен, может быть, потому что её писал солдат. Но прочитать очень стоит, потому что мало что может быть так ценно, как новая точка зрения на нечто хорошо знакомое!
"""

In [445]:
n_sents = len(fa('[^\.\!\?]+[\.\!\?]', text))
n_sents

25

In [227]:
def cleanse(s):
    rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
    return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))

In [402]:
uncert = ['наверное?', 'может[\s-]?быть', 'кажеть?ся', 
          'видимо', 'возможно', 'по[\s-]?видимому', 
          'вероятно', 'должно[\s-]?быть','пожалуй', 'как[\s-]видно']
cert = ['очевидно','конечно','точно','совершенно',
        'не\s?сомненно','разумееть?ся']
quan = ['вс[её]x?','всегда','ни-?когда', 'постоянно', 
        'ник(?:то|ого|ому|ем)',
        'кажд(?:ый|ая|ой|ому?|ое|ого|ую|ые|ою|ыми?|ых)',
        'всяк(?:ий|ая|ое|ого|ую|ому?|ой|ою|ими?|их|ие)',
        'люб(?:ой|ая|ое|ого|ому?|ую|ой|ыми?|ых|ые)']
imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
         'надо\W', 'нуж(?:но|ен|на|ны)', 
         'требуеть?ся', 'необходим(?:а|ы|о|)\W']
fa('|'.join(imper), text)

['обязана']

In [170]:
allpos= ['PRED', 'None', 'PRTS', 'ADJF', 'INFN', 
         'PRTF', 'NOUN', 'ADVB', 'VERB', 'NPRO', 
         'NUMR', 'CONJ', 'ADJS', 'PRCL', 'PREP', 'COMP', 'INTJ']

In [461]:
def extract_features(text, 
                     morph=pymorphy2.MorphAnalyzer(), 
                     pos_types=['ADJF', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP', 'INTJ'],
                     uncert = ['наверное?', 'может[\s-]?быть', 'кажеть?ся', 
                               'видимо', 'возможно', 'по[\s-]?видимому', 
                               'вероятно', 'должно[\s-]?быть','пожалуй', 'как[\s-]?видно'],
                     cert = ['очевидно','конечно','точно','совершенно',
                             'не\s?сомненно','разумееть?ся', 
                             'по[\s-]?любому','сто[\s-]?пудово?'],
                     quan = ['вс[её]x?','всегда','ни-?когда', 'постоянно', 
                             'ник(?:то|ого|ому|ем)', 
                             'кажд(?:ый|ая|ой|ому?|ое|ого|ую|ые|ою|ыми?|ых)',
                             'всяк(?:ий|ая|ое|ого|ую|ому?|ой|ою|ими?|их|ие)',
                             'люб(?:ой|ая|ое|ого|ому?|ую|ой|ыми?|ых|ые)'],
                     imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
                              'надо\W', 'нуж(?:но|ен|на|ны)', 
                              'требуеть?ся', 'необходим(?:а|ы|о|)\W'],
                    ):
    
    #length in chars and words
    len_char = len(text)
    len_word = len(text.split())
    len_sent = len(fa('[^\.\!\?]+[\.\!\?]', text))
    pun = fa('[\.+,!\?:-]',text)
    n_pun = len(pun)
    braсket_list = fa('[\(\)]',text)
      
    #POS & grammem
    def cleanse(s):
        rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
        return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))
    
    def parse_text(text, morph=morph):
        tokens = cleanse(text).split()
        return [morph.parse(t) for t in tokens]
    
    parsed_text = parse_text(text)
    pos_list = [str(p[0].tag.POS) for p in parsed_text]
    n_nouns = len([t for t in pos_list if t=='NOUN'])
    n_verbs = len([t for t in pos_list if t=='VERB'])
    anim_list = [str(p[0].tag.animacy) for p in parsed_text]
    pers_list = [str(p[0].tag.person) for p in parsed_text]
    tns_list = [str(p[0].tag.tense) for p in parsed_text]
    asp_list = [str(p[0].tag.aspect) for p in parsed_text]
      
    r = lambda x: round(x, 5)
    
    features = {
        #surface features
        'len_char': len_char, 
        'len_word': len_word,
        'len_sent': len_sent,
        'm_len_word': r(len_char / len_word),
        'm_len_sent': r(len_word / len_sent),
        #punctuation
        'p_pun': r(len(pun) / len_char),
        'p_dot': r(len([i for i in pun if i=='.']) / len(pun)),
        'p_qm': r(len([i for i in pun if i=='?']) / len(pun)),
        'p_excl': r(len([i for i in pun if i=='!']) / len(pun)),
        'p_comma': r(len([i for i in pun if i==',']) / len(pun)),
        'p_brkt': r(len(braсket_list) / len_char),
        'p_brkt_up': r(len([i for i in braсket_list if i==')']) / len(braсket_list)),
        #POS form
        #'pos_form': ' '.join(pos_list),
        'pos_richness': len(set(pos_list)),
        #grammem features
        'p_anim': r(len([t for t in anim_list if t=='anim']) / n_nouns),
        'p_1per': r(len([t for t in pers_list if t=='1per']) / n_verbs),
        'p_3per': r(len([t for t in pers_list if t=='3per']) / n_verbs),
        'p_past': r(len([t for t in tns_list if t=='past']) / n_verbs),
        #'p_fut': r(len([t for t in tns_list if t=='futr']) / n_verbs),
        'p_pres': r(len([t for t in tns_list if t=='pres']) / n_verbs),
        'p_perf': r(len([t for t in asp_list if t=='perf']) / n_verbs),
        #lexical features
        'p_uncert': r(len(fa('|'.join(uncert), text.lower())) / len_word),
        'p_cert': r(len(fa('|'.join(cert), text.lower())) / len_word),
        'p_quan': r(len(fa('|'.join(quan), text.lower())) / len_word),
        'p_imper': r(len(fa('|'.join(imper), text.lower())) / len_word),
        
    }
    
    for f in pos_types:
        features['p_'+f] = r(len([t for t in pos_list if t==f])/len(pos_list))
        
    return features

In [462]:
%%time
extract_features(text)

Wall time: 31.2 ms


{'len_char': 1848,
 'len_sent': 25,
 'len_word': 292,
 'm_len_sent': 11.68,
 'm_len_word': 6.32877,
 'p_1per': 0.4,
 'p_3per': 0.43333,
 'p_ADJF': 0.11263,
 'p_ADVB': 0.13311,
 'p_CONJ': 0.10239,
 'p_INTJ': 0.0,
 'p_NOUN': 0.24232,
 'p_PREP': 0.10239,
 'p_VERB': 0.10239,
 'p_anim': 0.23944,
 'p_brkt': 0.00108,
 'p_brkt_up': 0.5,
 'p_cert': 0.00342,
 'p_comma': 0.47541,
 'p_dot': 0.39344,
 'p_excl': 0.01639,
 'p_imper': 0.00342,
 'p_past': 0.56667,
 'p_perf': 0.5,
 'p_pres': 0.53333,
 'p_pun': 0.03301,
 'p_qm': 0.0,
 'p_quan': 0.02397,
 'p_uncert': 0.03082,
 'pos_richness': 16}

In [460]:
len(extract_features(text))

30